In [1]:
!pip install pandasai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 14.4 MB/s eta 0:00:00m eta 0:00:010:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 13.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.6 MB/s eta 0:00:00m eta 0:00:010:01:01
  Created wheel for pandas: filename=pandas-1.5.3-cp312-cp312-linux_x86_64.whl size=11477855 sha256=153c0d06333cf1978b3e803964a99fab454ad77304db7c4767ddb1c6fa333073
  Stored in directory: /home/bachngo/.cache/pip/wheels/fb/83/18/8e7307aa1185c5498c5490e4d9c8a1732d9f1056e86c3491c6
Successfully built pandas
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolve

In [12]:
!pip install seaborn

zsh:1: no matches found: pandasai[extra-dependency-name]


In [1]:
from pandasai import SmartDataframe
from pandasai.llm.local_llm import LocalLLM
from pandasai.llm import OpenAI

In [2]:
# llm = LocalLLM(
#     api_base="https://api.groq.com/openai/v1",
#     model="llama3-8b-8192"
# )

llm = OpenAI(
    api_token="",
)

In [3]:
import pandas as pd
from pandasai import Agent

df = pd.read_csv("./data/train.csv")
agent = Agent(df, config={"llm": llm})

In [7]:
print(agent.chat("What is the feature with highest correlation in this data"))

The feature with the highest correlation with 'Microwave' is 'Gymnasium'.


<string>:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.


# Llama_index approach

In [17]:
import pandas as pd
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.llms.groq import Groq


llm = Groq(model="llama3-70b-8192")

data_df = pd.read_csv("../data/train.csv")
query_engine = PandasQueryEngine(df=data_df, verbose=True, llm=llm)

In [36]:
# response = query_engine.query("Draw the data correlation matrix")

In [30]:
print(response.response)

There was an error running the output as Python code. Error message: could not convert string to float: 'Hooghly Chinsurah'


In [31]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="pandas_engine",
            description=(
                "Provides pandas code and execution on the data frame based on the query. "
            ),
        ),)
]

In [35]:
# # setup pack arguments
# from llama_index.core.agent import AgentRunner
# from llama_index.agent.llm_compiler.step import LLMCompilerAgentWorker

# agent_worker = LLMCompilerAgentWorker.from_tools(
#     tools, llm=llm, verbose=True
# )
# agent = AgentRunner(agent_worker)

# # start using the agent
# response = agent.chat("Perform analytics on the data frame")

In [48]:
from typing import Any, Dict, Optional, List
import re

def parse_code_markdown(text: str, only_last: bool) -> List[str]:
    # Regular expression pattern to match code within triple-backticks with an optional programming language
    pattern = r"```[a-zA-Z]*\n(.*?)```"

    # Find all matches of the pattern in the text
    matches = re.findall(pattern, text, re.DOTALL)

    # Return the last matched group if requested
    code = matches[-1] if matches and only_last else matches

    # If empty, we optimistically assume the output is the code
    if not code:
        # Strip the text to handle cases where the code may start or end with triple backticks or quotes
        candidate = text.strip()

        # Handle cases where the code is surrounded by regular quotes
        if candidate.startswith('"') and candidate.endswith('"'):
            candidate = candidate[1:-1]

        if candidate.startswith("'") and candidate.endswith("'"):
            candidate = candidate[1:-1]

        if candidate.startswith("`") and candidate.endswith("`"):
            candidate = candidate[1:-1]

        # Handle triple backticks at the start
        if candidate.startswith("```"):
            candidate = re.sub(r"^```[a-zA-Z]*\n?", "", candidate)

        # Handle triple backticks at the end
        if candidate.endswith("```"):
            candidate = candidate[:-3]

        code = [candidate.strip()]

    return code

In [53]:
# Example usage
text = """```python
df.describe(include='all').to_string() + '\n' + df.info() + '\n' + 'Correlation Matrix:\n' + df.corr().to_string() + '\n' + 'Missing Values:\n' + df.isnull().sum().to_string() + '\n' + 'Value Counts:\n' + df.apply(pd.value_counts).to_string()
```"""

parse_code_markdown(text, only_last=True)

"df.describe(include='all').to_string() + '\n' + df.info() + '\n' + 'Correlation Matrix:\n' + df.corr().to_string() + '\n' + 'Missing Values:\n' + df.isnull().sum().to_string() + '\n' + 'Value Counts:\n' + df.apply(pd.value_counts).to_string()\n"

In [50]:
df = data_df
df.head().to_string()


"      id  Area           Location  No. of Bedrooms  Resale  MaintenanceStaff  Gymnasium  SwimmingPool  LandscapedGardens  JoggingTrack  RainWaterHarvesting  IndoorGames  ShoppingMall  Intercom  SportsFacility  ATM  ClubHouse  School  24X7Security  PowerBackup  CarParking  StaffQuarter  Cafeteria  MultipurposeRoom  Hospital  WashingMachine  Gasconnection  AC  Wifi  Children'splayarea  LiftAvailable  BED  VaastuCompliant  Microwave  GolfCourse  TV  DiningTable  Sofa  Wardrobe  Refrigerator     Price\n0  26813   964  Hooghly Chinsurah                3       0                 9          9             9                  9             9                    9            9             9         9               9    9          9       9             9            9           9             9          9                 9         9               9              9   9     9                   9              9    9                9          9           9   9            9     9         9             9   

In [59]:
def default_output_processor(output: str, df: pd.DataFrame, timeout: int = 10, **output_kwargs: Any) -> str:
    """Process outputs in a default manner with a timeout."""
    if sys.version_info < (3, 9):
        logger.warning(
            "Python version must be >= 3.9 in order to use "
            "the default output processor, which executes "
            "the Python query. Instead, we will return the "
            "raw Python instructions as a string."
        )
        return output

    local_vars = {"df": df, "sns": sns, "plt": plt, "np": np, "pd": pd}
    global_vars = globals()  # Access global variables

    output = parse_code_markdown(output, only_last=True)

    try:
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(timeout)
        
        tree = ast.parse(output)
        module = ast.Module(tree.body[:-1], type_ignores=[])
        exec(ast.unparse(module), global_vars, local_vars)  # type: ignore
        module_end = ast.Module(tree.body[-1:], type_ignores=[])
        module_end_str = ast.unparse(module_end)  # type: ignore
        
        if module_end_str.strip("'\"") != module_end_str:
            # if there's leading/trailing quotes, then we need to eval
            # string to get the actual expression
            module_end_str = eval(module_end_str, global_vars, local_vars)
        
        try:
            # str(pd.dataframe) will truncate output by display.max_colwidth
            # set width temporarily to extract more text
            output_str = str(eval(module_end_str, global_vars, local_vars))
            
            if show_plot() == Status.SHOW_PLOT_SUCCESS:
                return "Plot displayed successfully to the user!"
            
            return output_str

        except Exception:
            raise
    except TimeoutException:
        return "The execution timed out. Please try again with optimized code or increase the timeout limit."
    except Exception as e:
        err_string = (
            "There was an error running the output as Python code. "
            f"Error message: {e}"
        )
        traceback.print_exc()
        return err_string
    finally:
        signal.alarm(0)  # Disable the alarm

No result variable defined
No plot generated


In [2]:
import re
import io
import logging
import signal
import sys
import ast
import re
import traceback

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from enum import Enum
from typing import Any, Dict, Optional, List


logger = logging.getLogger(__name__)

class TimeoutException(Exception):
    pass


class Status(Enum):
    NO_PLOT = "No plot"
    SHOW_PLOT_SUCCESS = "Show plot successfully!"
    SHOW_PLOT_FAILED = "Show plot failed!"
    
def parse_code_markdown(text: str, only_last: bool) -> List[str]:
    # Regular expression pattern to match code within triple-backticks with an optional programming language
    pattern = r"```[a-zA-Z]*\n(.*?)```"

    # Find all matches of the pattern in the text
    matches = re.findall(pattern, text, re.DOTALL)

    # Return the last matched group if requested
    code = matches[-1] if matches and only_last else matches

    # If empty, we optimistically assume the output is the code
    if not code:
        # Strip the text to handle cases where the code may start or end with triple backticks or quotes
        candidate = text.strip()

        # Handle cases where the code is surrounded by regular quotes
        if candidate.startswith('"') and candidate.endswith('"'):
            candidate = candidate[1:-1]

        if candidate.startswith("'") and candidate.endswith("'"):
            candidate = candidate[1:-1]

        if candidate.startswith("`") and candidate.endswith("`"):
            candidate = candidate[1:-1]

        # Handle triple backticks at the start
        if candidate.startswith("```"):
            candidate = re.sub(r"^```[a-zA-Z]*\n?", "", candidate)

        # Handle triple backticks at the end
        if candidate.endswith("```"):
            candidate = candidate[:-3]

        code = candidate.strip()

    return code

# def show_plot() -> str:
#     try:
#         # Ensure there's a plot to display
#         if not plt.get_fignums():
#             return Status.NO_PLOT
        
#         # Create an in-memory bytes buffer for the plot image
#         buffer = io.BytesIO()
#         plt.savefig(buffer, format='png')
#         buffer.seek(0)
        
#         # Clear the plot
#         plt.close()
        
#         # Display the plot image
#         image = cl.Image(
#             name="plot", 
#             size="large", 
#             display="inline", 
#             content=buffer.getvalue())
#         run_sync(cl.Message(
#             content="",
#             elements=[image]
#         ).send())
        
#         return Status.SHOW_PLOT_SUCCESS
#     except Exception as e:
#         return Status.SHOW_PLOT_FAILED


def timeout_handler(signum, frame):
    raise TimeoutException("Timed out!")

def default_output_processor(output: str, df: pd.DataFrame, timeout: int = 10, **output_kwargs: Any) -> str:
    """Process outputs in a default manner with a timeout."""
    if sys.version_info < (3, 9):
        logger.warning(
            "Python version must be >= 3.9 in order to use "
            "the default output processor, which executes "
            "the Python query. Instead, we will return the "
            "raw Python instructions as a string."
        )
        return output

    local_vars = {"df": df, "sns": sns, "plt": plt, "np": np, "pd": pd}
    global_vars = {}

    output = parse_code_markdown(output, only_last=True)

    try:
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(timeout)
        
        tree = ast.parse(output)
        module = ast.Module(tree.body[:-1], type_ignores=[])
        exec(ast.unparse(module), {}, local_vars)  # type: ignore
        module_end = ast.Module(tree.body[-1:], type_ignores=[])
        module_end_str = ast.unparse(module_end)  # type: ignore
        
        if module_end_str.strip("'\"") != module_end_str:
            # if there's leading/trailing quotes, then we need to eval
            # string to get the actual expression
            module_end_str = eval(module_end_str, global_vars, local_vars)
        
        try:
            # str(pd.dataframe) will truncate output by display.max_colwidth
            # set width temporarily to extract more text
            output_str = str(eval(module_end_str, global_vars, local_vars))
            
            # if show_plot() == Status.SHOW_PLOT_SUCCESS:
            #     return "Plot displayed successfully to the user!"
            
            return output_str

        except Exception:
            raise
    except TimeoutException:
        return "The execution timed out. Please try again with optimized code or increase the timeout limit."
    except Exception as e:
        err_string = (
            "There was an error running the output as Python code. "
            f"Error message: {e}"
        )
        traceback.print_exc()
        return err_string
    finally:
        signal.alarm(0)  # Disable the alarm

In [3]:
df = pd.read_csv("../data/train.csv")

sample_output ="""
```python
df['C'] = df['A'] + df['B']
df
```
"""



default_output_processor(sample_output, df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26370 entries, 0 to 26369
Data columns (total 41 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   26370 non-null  int64 
 1   Area                 26370 non-null  int64 
 2   Location             26370 non-null  object
 3   No. of Bedrooms      26370 non-null  int64 
 4   Resale               26370 non-null  int64 
 5   MaintenanceStaff     26370 non-null  int64 
 6   Gymnasium            26370 non-null  int64 
 7   SwimmingPool         26370 non-null  int64 
 8   LandscapedGardens    26370 non-null  int64 
 9   JoggingTrack         26370 non-null  int64 
 10  RainWaterHarvesting  26370 non-null  int64 
 11  IndoorGames          26370 non-null  int64 
 12  ShoppingMall         26370 non-null  int64 
 13  Intercom             26370 non-null  int64 
 14  SportsFacility       26370 non-null  int64 
 15  ATM                  26370 non-null  int64 
 16  Club

Traceback (most recent call last):
  File "/tmp/ipykernel_51311/824892047.py", line 133, in default_output_processor
    output_str = str(eval(module_end_str, global_vars, local_vars))
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
TypeError: can only concatenate str (not "NoneType") to str


'There was an error running the output as Python code. Error message: can only concatenate str (not "NoneType") to str'